In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Drive y lectura de datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#put your own path in google drive
%cd "/content/drive/MyDrive/7 semestre/Reto1/Modelo"
!ls

/content/drive/MyDrive/7 semestre/Reto1/Modelo
dfhotstand.csv	     modelo_RandomF_Lasso.ipynb  test.csv
dflabelstand.csv     RegressionTree.ipynb	 test_final
Limpieza_test.ipynb  SVM.ipynb			 test.gsheet


# Limpieza

## Label

In [13]:
test = pd.read_csv("/content/drive/MyDrive/7 semestre/Reto1/Modelo/test.csv")
id = test["Id"]
test = test.drop(["Id", "LotFrontage"], axis=1)

test['GarageYrBlt'] = test['GarageYrBlt'].replace('No', np.nan)
test['GarageYrBlt'] = test['GarageYrBlt'].astype(float)
test['GarageYrBlt'] = test['GarageYrBlt'].fillna(0).astype(int)

test['Alley'] = test['Alley'].astype(str).str.strip().replace('nan', 'No')

test['MasVnrType'] = test['MasVnrType'].astype(str).str.strip().replace(['None', 'nan'], 'No')

test['BsmtQual'] = test['BsmtQual'].apply(lambda a:100 if a == 'Ex' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:95 if a == 'Gd' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:85 if a == 'TA' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:75 if a == 'Fa' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:70 if a == 'Po' else a)
test["BsmtQual"] = test["BsmtQual"].fillna(0)
test['BsmtQual'] = test['BsmtQual']. astype(int)
test["BsmtCond"] = test["BsmtCond"].fillna("No")
test["BsmtExposure"] = test["BsmtExposure"].fillna("No")
test["BsmtFinType1"] = test["BsmtFinType1"].fillna("No")
test["BsmtFinType2"] = test["BsmtFinType2"].fillna("No")
test['FireplaceQu'] = test['FireplaceQu'].astype(str).str.strip().replace('nan', 'No')
test['GarageType'] = test['GarageType'].astype(str).str.strip().replace('nan', 'No')
test['GarageFinish'] = test['GarageFinish'].astype(str).str.strip().replace('nan', 'No')
test['GarageQual'] = test['GarageQual'].astype(str).str.strip().replace('nan', 'No')
test['PoolQC'] = test['PoolQC'].astype(str).str.strip().replace('nan', 'No')

# Op 1

op1 = test.copy()

# Label encoding
op1["Utilities"] = op1["Utilities"].replace({'AllPub': 4, 'NoSewr': 3,'NoSeWa': 2,'ELO': 1})
op1["ExterQual"] = op1["ExterQual"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["ExterCond"] = op1["ExterCond"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["BsmtCond"] = op1["BsmtCond"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["BsmtExposure"] = op1["BsmtExposure"].replace({'Gd': 3, 'Av': 2,'Mn': 1,'No': 0})
op1["BsmtFinType1"] = op1["BsmtFinType1"].replace({'GLQ': 6, 'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'No': 0})
op1["BsmtFinType2"] = op1["BsmtFinType2"].replace({'GLQ': 6, 'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'No': 0})
op1["HeatingQC"] = op1["HeatingQC"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0})
op1["CentralAir"] = op1["CentralAir"].replace({'Y': 1, 'N': 0})
op1["KitchenQual"] = op1["KitchenQual"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["FireplaceQu"] = op1["FireplaceQu"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["GarageQual"] = op1["GarageQual"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["GarageCond"] = op1["GarageCond"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["PoolQC"] = op1["PoolQC"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'No': 0})



# Hot encoding
categorical_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical','Functional', 'GarageType', 'GarageFinish','PavedDrive','Fence','MiscFeature','SaleType','SaleCondition']
op1 = pd.get_dummies(op1, columns=categorical_cols)
op1['RoofMatl_ClyTile'] = 0

# Estandarización
variables_no_estandarizar = ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_No', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofStyle_Flat', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_ClyTile', 'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng', 'Exterior1st_WdShing', 'Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng', 'MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_No', 'MasVnrType_Stone', 'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'Heating_Floor', 'Heating_GasA', 'Heating_GasW', 'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr', 'Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_No', 'GarageFinish_Fin', 'GarageFinish_No', 'GarageFinish_RFn', 'GarageFinish_Unf', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'Fence_GdPrv', 'Fence_GdWo', 'Fence_MnPrv', 'Fence_MnWw', 'Fence_No', 'MiscFeature_Gar2', 'MiscFeature_No', 'MiscFeature_Othr', 'MiscFeature_Shed', 'MiscFeature_TenC', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial']

scaler = StandardScaler()
columnas_a_estandarizar = [col for col in op1.columns if col not in variables_no_estandarizar]
op1[columnas_a_estandarizar] = scaler.fit_transform(op1[columnas_a_estandarizar])
op1_Kaggle = pd.DataFrame(op1)

# Eliminación de columnas
op1_Kaggle = op1_Kaggle.drop(['FireplaceQu', 'GarageYrBlt', 'GarageCond', 'GarageArea', 'PoolArea', 'MiscFeature_Gar2',
                                  'Neighborhood_Somerst', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard',
                                  'Exterior2nd_MetalSd', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng',
                                  'SaleCondition_Partial'], axis=1)



# Selección de variables significantes
significant_variables_list = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtExposure', 'BsmtFinSF1',
                              'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'GrLivArea', 'BsmtFullBath', 'HalfBath', 'KitchenAbvGr',
                              'KitchenQual', 'Fireplaces', 'GarageCars', 'GarageQual', 'WoodDeckSF', 'ScreenPorch', 'MSZoning_FV',
                              'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'LotConfig_Inside', 'LandSlope_Sev', 'Neighborhood_Crawfor',
                              'Neighborhood_Edwards', 'Neighborhood_MeadowV', 'Neighborhood_NridgHt', 'Neighborhood_StoneBr',
                              'Condition1_Artery', 'Condition1_Feedr', 'Condition1_RRAe', 'Condition2_PosA', 'Condition2_PosN',
                              'BldgType_Twnhs', 'BldgType_TwnhsE', 'RoofMatl_ClyTile', 'RoofMatl_CompShg', 'Exterior1st_BrkComm',
                              'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior2nd_BrkFace', 'Foundation_BrkTil',
                              'Foundation_CBlock', 'Foundation_Wood', 'Heating_GasA', 'Heating_Grav', 'Electrical_SBrkr',
                              'Functional_Maj1', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Typ',
                              'GarageType_2Types', 'SaleType_COD', 'SaleType_ConLI', 'SaleType_WD', 'SaleCondition_Normal']

op1_Kaggle = op1_Kaggle[significant_variables_list]

In [16]:
# Conteo de las variables faltantes
op1_Kaggle.isnull().sum()/op1_Kaggle.shape[0]*100 #porcentaje

LotArea                 0.0
OverallQual             0.0
OverallCond             0.0
YearBuilt               0.0
YearRemodAdd            0.0
BsmtExposure            0.0
BsmtFinSF1              0.0
TotalBsmtSF             0.0
HeatingQC               0.0
CentralAir              0.0
GrLivArea               0.0
BsmtFullBath            0.0
HalfBath                0.0
KitchenAbvGr            0.0
KitchenQual             0.0
Fireplaces              0.0
GarageCars              0.0
GarageQual              0.0
WoodDeckSF              0.0
ScreenPorch             0.0
MSZoning_FV             0.0
MSZoning_RH             0.0
MSZoning_RL             0.0
MSZoning_RM             0.0
LotConfig_Inside        0.0
LandSlope_Sev           0.0
Neighborhood_Crawfor    0.0
Neighborhood_Edwards    0.0
Neighborhood_MeadowV    0.0
Neighborhood_NridgHt    0.0
Neighborhood_StoneBr    0.0
Condition1_Artery       0.0
Condition1_Feedr        0.0
Condition1_RRAe         0.0
Condition2_PosA         0.0
Condition2_PosN     

In [15]:
op1_Kaggle["BsmtFinSF1"] = op1_Kaggle["BsmtFinSF1"].fillna(op1_Kaggle["BsmtFinSF1"].median())
op1_Kaggle["TotalBsmtSF"] = op1_Kaggle["TotalBsmtSF"].fillna(op1_Kaggle["TotalBsmtSF"].median())
op1_Kaggle["BsmtFullBath"] = op1_Kaggle["BsmtFullBath"].fillna(op1_Kaggle["BsmtFullBath"].median())
op1_Kaggle["KitchenQual"] = op1_Kaggle["KitchenQual"].fillna(op1_Kaggle["KitchenQual"].median())
op1_Kaggle["GarageCars"] = op1_Kaggle["GarageCars"].fillna(op1_Kaggle["GarageCars"].median())

In [17]:
# Guardar el DataFrame en un archivo CSV
op1_Kaggle.to_csv('/content/drive/MyDrive/7 semestre/Reto1/Modelo/test_final/test_op1_Kaggle.csv', index=False)

## Hot

In [18]:
# Op 2

op2 = test.copy()

# Hot encoding
categoricas = op2.select_dtypes(include='object').columns
numericas = op2.select_dtypes(include='number').columns

categoricas = list(categoricas)
op2 = pd.get_dummies(op2, columns=categoricas)

# Estandarización
variables_no_estandarizar = ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
                             'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl',
                             'Alley_No', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2',
                             'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS',
                             'LandContour_Low', 'LandContour_Lvl', 'LotConfig_Corner',
                             'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl',
                             'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
                             'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
                             'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
                             'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
                             'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber',
                             'Neighborhood_Veenker', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN',
                             'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr',
                             'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_1Fam',
                             'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf',
                             'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl',
                             'RoofStyle_Flat', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_ClyTile',
                             'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng', 'Exterior1st_WdShing', 'Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng', 'MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_No', 'MasVnrType_Stone', 'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'Heating_Floor', 'Heating_GasA', 'Heating_GasW', 'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr', 'Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_No', 'GarageFinish_Fin', 'GarageFinish_No', 'GarageFinish_RFn', 'GarageFinish_Unf', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'Fence_GdPrv', 'Fence_GdWo', 'Fence_MnPrv', 'Fence_MnWw', 'Fence_No', 'MiscFeature_Gar2', 'MiscFeature_No', 'MiscFeature_Othr', 'MiscFeature_Shed', 'MiscFeature_TenC', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial']


scaler = StandardScaler()
columnas_a_estandarizar = [col for col in op2.columns if col not in variables_no_estandarizar]
op2[columnas_a_estandarizar] = scaler.fit_transform(op2[columnas_a_estandarizar])
op2_Kaggle = pd.DataFrame(op2)

# Eliminación de columnas
op2['Condition2_RRAn'] = 0
op2['Condition2_RRNn'] = 0
op2['HouseStyle_2.5Fin'] = 0
op2['RoofMatl_Membran'] = 0
op2['RoofMatl_Metal'] = 0
op2['RoofMatl_Roll'] = 0
op2['Electrical_Mix'] = 0
op2['GarageQual_Ex'] = 0


op2_Kaggle = op2_Kaggle.drop(['Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_MetalSd',
                              'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'GarageCars', 'SaleCondition_Partial', 'MiscFeature_Gar2',
                              'Neighborhood_Somerst', 'BsmtFinType1_No', 'BsmtFinType2_No',  'GarageFinish_No', 'GarageQual_No'], axis=1)

# Selección de variables significantes
significant_variables_list = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF',
                              'GrLivArea', 'BsmtFullBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'WoodDeckSF',
                              'EnclosedPorch', 'ScreenPorch', 'PoolArea', 'MSZoning_C (all)', 'MSZoning_RM', 'Street_Pave',
                              'LandContour_Bnk', 'LandContour_Low', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Sev',
                              'Neighborhood_BrDale', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
                              'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NWAmes',
                              'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
                              'Neighborhood_StoneBr', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_RRAe', 'Condition2_Artery',
                              'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_RRAn', 'Condition2_RRNn',
                              'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_2.5Fin', 'RoofStyle_Flat', 'RoofStyle_Gable',
                              'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofMatl_CompShg', 'RoofMatl_Membran',
                              'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl',
                              'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior2nd_BrkFace', 'ExterCond_Gd',
                              'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_Wood', 'BsmtCond_Po', 'BsmtExposure_Gd', 'Heating_GasA',
                              'Heating_Grav', 'HeatingQC_Ex', 'CentralAir_N', 'Electrical_Mix', 'KitchenQual_Ex', 'Functional_Maj1',
                              'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Typ', 'GarageType_Attchd',
                              'GarageType_Detchd', 'GarageQual_Ex', 'GarageCond_Ex', 'GarageCond_Fa', 'PoolQC_No', 'Fence_GdWo',
                              'SaleType_COD', 'SaleType_ConLI', 'SaleType_WD', 'SaleCondition_Normal']

op2_Kaggle = op2_Kaggle[significant_variables_list]

<ipython-input-18-76af8e706133>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  op2['Condition2_RRAn'] = 0
<ipython-input-18-76af8e706133>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  op2['Condition2_RRNn'] = 0
<ipython-input-18-76af8e706133>:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  o

In [27]:
# Conteo de las variables faltantes
op2_Kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 94 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LotArea               1459 non-null   float64
 1   OverallQual           1459 non-null   float64
 2   OverallCond           1459 non-null   float64
 3   YearBuilt             1459 non-null   float64
 4   YearRemodAdd          1459 non-null   float64
 5   BsmtFinSF1            1459 non-null   float64
 6   TotalBsmtSF           1459 non-null   float64
 7   GrLivArea             1459 non-null   float64
 8   BsmtFullBath          1459 non-null   float64
 9   KitchenAbvGr          1459 non-null   float64
 10  TotRmsAbvGrd          1459 non-null   float64
 11  Fireplaces            1459 non-null   float64
 12  GarageArea            1459 non-null   float64
 13  WoodDeckSF            1459 non-null   float64
 14  EnclosedPorch         1459 non-null   float64
 15  ScreenPorch          

In [26]:
op2_Kaggle["BsmtFinSF1"] = op2_Kaggle["BsmtFinSF1"].fillna(op2_Kaggle["BsmtFinSF1"].median())
op2_Kaggle["TotalBsmtSF"] = op2_Kaggle["TotalBsmtSF"].fillna(op2_Kaggle["TotalBsmtSF"].median())
op2_Kaggle["BsmtFullBath"] = op2_Kaggle["BsmtFullBath"].fillna(op2_Kaggle["BsmtFullBath"].median())

#op2_Kaggle["KitchenQual"] = op2_Kaggle["KitchenQual"].fillna(op2_Kaggle["KitchenQual"].median())
#op2_Kaggle["GarageCars"] = op2_Kaggle["GarageCars"].fillna(op2_Kaggle["GarageCars"].median())

op2_Kaggle["GarageArea"] = op2_Kaggle["GarageArea"].fillna(op2_Kaggle["GarageArea"].median())

In [28]:
# Guardar el DataFrame en un archivo CSV
op2_Kaggle.to_csv('/content/drive/MyDrive/7 semestre/Reto1/Modelo/test_final/test_op2_Kaggle.csv', index=False)